Importamos las librerias para el proyecto

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

Importamos el archivo Train y procedemos a realizar el trabajo de limpieza(etl).

In [2]:
df_train = pd.read_parquet('Datathon-main/train.parquet')

df_train.head(2)

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7048013474,https://boise.craigslist.org/apa/d/very-nice-b...,boise,https://boise.craigslist.org,1350,house,1200,2,2.0,1,...,0,0,0,w/d in unit,detached garage,https://images.craigslist.org/00B0B_cPiJMEheZe...,Super cute row house in the Boise bench area. ...,43.5851,-116.225,id
1,7043931179,https://cosprings.craigslist.org/apa/d/colorad...,colorado springs,https://cosprings.craigslist.org,1115,apartment,694,1,1.0,1,...,0,0,0,w/d in unit,carport,https://images.craigslist.org/00R0R_5XAoSKvfrz...,Windtree Apartment Homes currently has a spaci...,38.9137,-104.780,co


Realizamos una copia del dataframe original para poder hacer todos los cambios sin perder el original

In [9]:
df_train2 = df_train
len(df_train2) #verificamos que la cantidad de filas sea la correcta

346479

In [10]:
#Borramos los duplicados de la columna Description ya que asumimos que si la descipcion es identica se refiere a la misma vivienda y esto nos podria traer problemas en los modelos
df_train2 = df_train2.drop_duplicates('description')
len(df_train2)#vemos la cantidad de registro con los que vamos a continuar el trabajo

254991

In [41]:
print(df_train2.isnull().sum()) #verificamos la cantidad de nulos

id                 0
url                0
region_url         0
price              0
sqfeet             0
                  ..
unknown            0
attached garage    0
street parking     0
no parking         0
valet parking      0
Length: 88, dtype: int64


In [12]:
#completamos los valores null con empty
df_train2 = df_train2.fillna({'laundry_options': 'empty','parking_options': 'empty','description': 'empty' })

In [13]:
#vemos las columnas
df_train2.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'type', 'sqfeet', 'beds',
       'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished',
       'laundry_options', 'parking_options', 'image_url', 'description', 'lat',
       'long', 'state', 'category_price'],
      dtype='object')

In [20]:
#creamos la columna evaluatoria 
df_train2['category_binario'] = 0
df_train2['category_binario'] = np.where(df_train2['price']<=999 , 1 ,df_train2['category_binario'])
df_train2['category_binario'] = np.where(df_train2['price']> 999 , 0 ,df_train2['category_binario'])

In [22]:
df_train2['category_binario'].value_counts() #verficamos la cantidad de 0 y 1

0    149456
1    105535
Name: category_binario, dtype: int64

Comenzamos a crear dummis para las columnas que nos ayudaran a mejorar la exactitud del modelo

In [23]:
df_train2.state.unique()    

array(['id', 'co', 'va', 'az', 'tn', 'tx', 'ma', 'ks', 'pa', 'me', 'ms',
       'la', 'mn', 'ky', 'wa', 'al', 'ca', 'ga', 'nc', 'md', 'ia', 'fl',
       'or', 'ar', 'hi', 'nj', 'ut', 'wv', 'ny', 'sc', 'oh', 'mo', 'mi',
       'ak', 'sd', 'ok', 'dc', 'in', 'ct', 'ri', 'ne', 'nd', 'nv', 'wi',
       'nh', 'de', 'il', 'nm', 'mt', 'vt', 'wy'], dtype=object)

In [24]:
#Creamos los dummis para State y borramos la columna 'state'
df_train2[['id', 'co', 'va', 'az', 'tn', 'tx', 'ma', 'ks', 'pa', 'me', 'ms',
       'la', 'mn', 'ky', 'wa', 'al', 'ca', 'ga', 'nc', 'md', 'ia', 'fl',
       'or', 'ar', 'hi', 'nj', 'ut', 'wv', 'ny', 'sc', 'oh', 'mo', 'mi',
       'ak', 'sd', 'ok', 'dc', 'in', 'ct', 'ri', 'ne', 'nd', 'nv', 'wi',
       'nh', 'de', 'il', 'nm', 'mt', 'vt', 'wy']] = pd.get_dummies(df_train2.state)
df_train2.drop(['state'],axis='columns', inplace=True)

In [42]:
df_train2.type.unique()

AttributeError: 'DataFrame' object has no attribute 'type'

In [28]:
# Creamos los dummis para type y borramos la columna 'type'
df_train2[['house', 'apartment', 'duplex', 'loft', 'condo', 'manufactured',
       'townhouse', 'in-law', 'cottage/cabin', 'flat', 'assisted living',
       'land']] = pd.get_dummies(df_train2.type)
df_train2.drop(['type'],axis='columns', inplace=True)

In [27]:
df_train.parking_options.unique()

array(['detached garage', 'carport', 'off-street parking', None,
       'attached garage', 'street parking', 'no parking', 'valet parking'],
      dtype=object)

In [29]:
# Creamos los dummis para parkimg options y borramos la columna 'parkimg_options'
df_train2[['detached garage', 'carport', 'off-street parking', 'unknown',
       'attached garage', 'street parking', 'no parking', 'valet parking']] = pd.get_dummies(df_train2.parking_options)
df_train2.drop(['parking_options'],axis='columns', inplace=True)

In [40]:
df_train2['category_binario'].dtype

dtype('int64')

Borramos mas columnas que verificamos que no nos sirven para el modelo

In [45]:
df_train2= df_train2.drop(columns=['id', 'url', 'region_url', 'image_url','description', 'laundry_options','smoking_allowed'])

In [30]:
df_train2 = df_train2.drop(columns=['region'])

In [31]:
df_train2 = df_train2.drop(columns=['lat','long'])

In [46]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Crear el pipeline


pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', DecisionTreeClassifier())
])

In [47]:
# Cargar los datos de entrada
X = df_train2.drop(columns=['price','category_binario'])
y = df_train2['category_binario']

In [48]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X , y , test_size=0.2)

In [49]:
# Ajustar el pipeline a los datos de entrenamiento
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', DecisionTreeClassifier())])

In [50]:
# Hacer predicciones en el conjunto de prueba
y_pred = pipe.predict(X_test)

In [51]:
# Evaluar la precisión del modelo
score = pipe.score(X_test, y_test)
print("Accuracy: ", score)

Accuracy:  0.8795466577776034


Importamos el dataset de test para realizar las pruebas 

In [85]:
df_test = pd.read_parquet('Datathon-main/test.parquet')

Vamos a realizar la creacion de los dummis en el archivo test

In [86]:
df_test[['id', 'co', 'va', 'az', 'tn', 'tx', 'ma', 'ks', 'pa', 'me', 'ms',
       'la', 'mn', 'ky', 'wa', 'al', 'ca', 'ga', 'nc', 'md', 'ia', 'fl',
       'or', 'ar', 'hi', 'nj', 'ut', 'wv', 'ny', 'sc', 'oh', 'mo', 'mi',
       'ak', 'sd', 'ok', 'dc', 'in', 'ct', 'ri', 'ne', 'nd', 'nv', 'wi',
       'nh', 'de', 'il', 'nm', 'mt', 'vt', 'wy']] = pd.get_dummies(df_test.state)
df_test.drop(['state'],axis='columns', inplace=True)

In [87]:
df_test.type.unique()

array(['manufactured', 'apartment', 'house', 'townhouse', 'condo',
       'duplex', 'cottage/cabin', 'loft', 'in-law', 'flat'], dtype=object)

In [88]:
df_test[['house', 'apartment', 'duplex', 'loft', 'condo', 'manufactured',
       'townhouse', 'in-law', 'cottage/cabin', 'flat']] = pd.get_dummies(df_test.type)
df_test.drop(['type'],axis='columns', inplace=True)

In [93]:
df_test = df_test.fillna({'parking_options' : 'vacio'})

In [90]:
df_test[['off-street parking', 'carport', 'attached garage', 'vacio',
       'street parking', 'detached garage', 'no parking', 'valet parking']] = pd.get_dummies(df_test.parking_options)
df_test.drop(['parking_options'],axis='columns', inplace=True)

In [95]:
df_test.vacio.unique()

array([0, 1], dtype=uint8)

vemos que las columnas que trae el archivo test no son las mismas que tenemos actualmente en train2, por lo cual vamos a proceder a borrar columnas
de ambos datasets para que sean iguales.

In [96]:
df_test.columns

Index(['id', 'url', 'region', 'region_url', 'sqfeet', 'beds', 'baths',
       'cats_allowed', 'dogs_allowed', 'smoking_allowed', 'wheelchair_access',
       'electric_vehicle_charge', 'comes_furnished', 'laundry_options',
       'image_url', 'description', 'lat', 'long', 'co', 'va', 'az', 'tn', 'tx',
       'ma', 'ks', 'pa', 'me', 'ms', 'la', 'mn', 'ky', 'wa', 'al', 'ca', 'ga',
       'nc', 'md', 'ia', 'fl', 'or', 'ar', 'hi', 'nj', 'ut', 'wv', 'ny', 'sc',
       'oh', 'mo', 'mi', 'ak', 'sd', 'ok', 'dc', 'in', 'ct', 'ri', 'ne', 'nd',
       'nv', 'wi', 'nh', 'de', 'il', 'nm', 'mt', 'vt', 'wy', 'house',
       'apartment', 'duplex', 'loft', 'condo', 'manufactured', 'townhouse',
       'in-law', 'cottage/cabin', 'flat', 'off-street parking', 'carport',
       'attached garage', 'vacio', 'street parking', 'detached garage',
       'no parking', 'valet parking'],
      dtype='object')

In [121]:
#Creamos una copia del dataset nuevamente para no perder nada de lo trabajado anteriormente.
df_train1 = df_train2

In [124]:
df_train1.columns

Index(['sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished',
       'category_price', 'category_binario', 'co', 'va', 'az', 'tn', 'tx',
       'ma', 'ks', 'pa', 'me', 'ms', 'la', 'mn', 'ky', 'wa', 'al', 'ca', 'ga',
       'nc', 'md', 'ia', 'fl', 'or', 'ar', 'hi', 'nj', 'ut', 'wv', 'ny', 'sc',
       'oh', 'mo', 'mi', 'ak', 'sd', 'ok', 'dc', 'in', 'ct', 'ri', 'ne', 'nd',
       'nv', 'wi', 'nh', 'de', 'il', 'nm', 'mt', 'vt', 'wy', 'house',
       'apartment', 'duplex', 'loft', 'condo', 'manufactured', 'townhouse',
       'in-law', 'cottage/cabin', 'flat', 'detached garage', 'carport',
       'off-street parking', 'unknown', 'attached garage', 'street parking',
       'no parking', 'valet parking'],
      dtype='object')

In [ ]:
#Procedemos a borrar las sigueintes columnas del dataset de training
df_train1 = df_train1.drop(columns='price')
df_train1 = df_train1.drop(columns='assisted living')
df_train1 = df_train1.drop(columns='land')


In [126]:
df_train1 = df_train1.drop(columns='category_price')

In [100]:
#Procedemos a borrar las siguientes columnas del dataset de Testing
df_test = df_test.drop(columns=['region_url'])
df_test = df_test.drop(columns=['id'])
df_test = df_test.drop(columns=['url'])
df_test = df_test.drop(columns=['region'])
df_test = df_test.drop(columns=['smoking_allowed'])
df_test = df_test.drop(columns=['laundry_options'])
df_test = df_test.drop(columns=['image_url'])
df_test = df_test.drop(columns=['description'])
df_test = df_test.drop(columns=['lat', 'long'])

In [101]:
df_test['category_binario'] = 0

In [120]:
df_train1.columns

Index(['sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'co',
       'va', 'az', 'tn', 'tx', 'ma', 'ks', 'pa', 'me', 'ms', 'la', 'mn', 'ky',
       'wa', 'al', 'ca', 'ga', 'nc', 'md', 'ia', 'fl', 'or', 'ar', 'hi', 'nj',
       'ut', 'wv', 'ny', 'sc', 'oh', 'mo', 'mi', 'ak', 'sd', 'ok', 'dc', 'in',
       'ct', 'ri', 'ne', 'nd', 'nv', 'wi', 'nh', 'de', 'il', 'nm', 'mt', 'vt',
       'wy', 'house', 'apartment', 'duplex', 'loft', 'condo', 'manufactured',
       'townhouse', 'in-law', 'cottage/cabin', 'flat', 'off-street parking',
       'carport', 'attached garage', 'unknown', 'street parking',
       'detached garage', 'no parking', 'valet parking', 'category_binario'],
      dtype='object')

In [117]:
df_test.columns

Index(['sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'co',
       'va', 'az', 'tn', 'tx', 'ma', 'ks', 'pa', 'me', 'ms', 'la', 'mn', 'ky',
       'wa', 'al', 'ca', 'ga', 'nc', 'md', 'ia', 'fl', 'or', 'ar', 'hi', 'nj',
       'ut', 'wv', 'ny', 'sc', 'oh', 'mo', 'mi', 'ak', 'sd', 'ok', 'dc', 'in',
       'ct', 'ri', 'ne', 'nd', 'nv', 'wi', 'nh', 'de', 'il', 'nm', 'mt', 'vt',
       'wy', 'house', 'apartment', 'duplex', 'loft', 'condo', 'manufactured',
       'townhouse', 'in-law', 'cottage/cabin', 'flat', 'off-street parking',
       'carport', 'attached garage', 'unknown', 'street parking',
       'detached garage', 'no parking', 'valet parking', 'category_binario'],
      dtype='object')

Acomodamos las columnas para que nos queden igual al data set de Test

In [127]:
df_train1 = df_train1[['sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed',
       'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'co',
       'va', 'az', 'tn', 'tx', 'ma', 'ks', 'pa', 'me', 'ms', 'la', 'mn', 'ky',
       'wa', 'al', 'ca', 'ga', 'nc', 'md', 'ia', 'fl', 'or', 'ar', 'hi', 'nj',
       'ut', 'wv', 'ny', 'sc', 'oh', 'mo', 'mi', 'ak', 'sd', 'ok', 'dc', 'in',
       'ct', 'ri', 'ne', 'nd', 'nv', 'wi', 'nh', 'de', 'il', 'nm', 'mt', 'vt',
       'wy', 'house', 'apartment', 'duplex', 'loft', 'condo', 'manufactured',
       'townhouse', 'in-law', 'cottage/cabin', 'flat', 'off-street parking',
       'carport', 'attached garage', 'unknown', 'street parking',
       'detached garage', 'no parking', 'valet parking', 'category_binario']]

In [106]:
df_test = df_test.rename(columns={'vacio':'unknown'})

In [129]:
# Cargar los datos de entrada
X = df_train1.drop(columns=['category_binario'])
y = df_train1['category_binario']

In [130]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X , y , test_size=0.2)

In [131]:
# Ajustar el pipeline a los datos de entrenamiento
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier', DecisionTreeClassifier())])

In [132]:
# Hacer predicciones en el conjunto de prueba
y_pred = pipe.predict(X_test)
# Evaluar la precisión del modelo
score = pipe.score(X_test, y_test)
print(score)

0.8814682640836095


In [133]:
x1 =df_test.drop(columns=['category_binario'])

paraexportar = pipe.predict(x1)
score2 = pipe.score(X_test, y_test)
print(score2)

0.8814682640836095


In [ ]:
DF = pd.DataFrame(paraexportar)
DF.to_csv("lucasrdrz.csv", header='pred', index=False)